***GENERATED CODE FOR dataprep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def calculateFormula(df, functionsData, applyOn):
    dfcp = df
    for functionData in functionsData:
        if functionData['useNext']:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
            else:
                dfcp['dataPrepColumn_Inprocess'] = calculateEquation(
                    dfcp, functionData['operator'], functionData['operand1'], functionData['operand2'])
        else:
            if 'dataPrepColumn_Inprocess' in list(dfcp.columns):
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData['operator'],
                                                            functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['dataPrepColumn_Inprocess'], functionData['operand2'])
                dfcp.drop('dataPrepColumn_Inprocess', axis=1)
            else:
                if functionData['outputColumn'] == "newColumn":
                    newColumnName = "New_" + \
                        functionData['operand1']+"_" + \
                        functionData['operator']+"_"+functionData['operand2']
                    dfcp[newColumnName] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
                else:
                    dfcp[functionData['outputColumn']] = calculateEquation(dfcp, functionData[
                        'operator'], functionData['operand1'], functionData['operand2'])
    return dfcp


def calculateEquation(df, operator, columnA, columnB):
    if (operator == "add"):
        return df[columnA].add(df[columnB])
    elif (operator == "sub"):
        return df[columnA].sub(df[columnB])
    elif (operator == "mul"):
        return df[columnA].mul(df[columnB])
    elif (operator == "div"):
        return df[columnA].div(df[columnB])
    elif (operator == "mod"):
        return df[columnA].mod(df[columnB])


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#sourcePreExecutionHook()

	bostontrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/TrainData/BostonTrain.csv', 'filename': 'BostonTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(bostontrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(bostontrain,spark,json.dumps( {"url": "/FileStore/platform/TrainData/BostonTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/TrainData/BostonTrain.csv", "data_source": "hdfs", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/6074448315c6097d355fa3431618380998/0part.csv", "requestRatio": 1.0, "totalRows": 506, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 14, "numberOfRows": 506, "duplicateRowCount": 0, "stats": [{"column": "CRIM", "alias": "CRIM", "generated": 0, "type": "real", "max": 88.9762, "min": 0.00632, "mean": 3.6135235573122535, "missing": 0.0, "stddev": 8.6}, {"column": "ZN", "alias": "ZN", "generated": 0, "type": "real", "max": 101.32, "min": 1.89, "mean": 22.500415019762936, "missing": 0.0, "stddev": 20.5}, {"column": "INDUS", "alias": "INDUS", "generated": 0, "type": "real", "max": 27.74, "min": 0.46, "mean": 11.136778656126504, "missing": 0.0, "stddev": 6.86}, {"column": "CHAS", "alias": "CHAS", "generated": 0, "type": "real", "max": 1.0, "min": 0.0, "mean": 0.0691699604743083, "missing": 0.0, "stddev": 0.25}, {"column": "NOX", "alias": "NOX", "generated": 0, "type": "real", "max": 0.871, "min": 0.385, "mean": 0.5546950592885372, "missing": 0.0, "stddev": 0.12}, {"column": "RM", "alias": "RM", "generated": 0, "type": "real", "max": 8.78, "min": 3.5610000000000004, "mean": 6.284634387351788, "missing": 0.0, "stddev": 0.7}, {"column": "AGE", "alias": "AGE", "generated": 0, "type": "real", "max": 100.0, "min": 2.9, "mean": 68.57490118577078, "missing": 0.0, "stddev": 28.15}, {"column": "DIS", "alias": "DIS", "generated": 0, "type": "real", "max": 12.1265, "min": 1.1296, "mean": 3.795042687747034, "missing": 0.0, "stddev": 2.11}, {"column": "RAD", "alias": "RAD", "generated": 0, "type": "real", "max": 24.0, "min": 1.0, "mean": 9.549407114624506, "missing": 0.0, "stddev": 8.71}, {"column": "TAX", "alias": "TAX", "generated": 0, "type": "real", "max": 711.0, "min": 187.0, "mean": 408.2371541501976, "missing": 0.0, "stddev": 168.54}, {"column": "PTRATIO", "alias": "PTRATIO", "generated": 0, "type": "real", "max": 22.0, "min": 12.6, "mean": 18.455533596837967, "missing": 0.0, "stddev": 2.16}, {"column": "B", "alias": "B", "generated": 0, "type": "real", "max": 396.9, "min": 0.32, "mean": 356.67403162055257, "missing": 0.0, "stddev": 91.29}, {"column": "LSTAT", "alias": "LSTAT", "generated": 0, "type": "real", "max": 37.97, "min": 1.73, "mean": 12.653063241106723, "missing": 0.0, "stddev": 7.14}, {"column": "MEDV", "alias": "MEDV", "generated": 0, "type": "real", "max": 50.0, "min": 5.0, "mean": 22.532806324110698, "missing": 0.0, "stddev": 9.2}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "formula", "applyOn": [{"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}, {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}], "functionsData": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "ZN", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}]}]}], "functionChanges": [{"columnName": "ZN", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "ZN", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}]}]}, {"columnName": "INDUS", "functionName": "Formula", "Type": "real", "Parameters": [{"operand1": "ZN", "operator": "add", "operand2": "INDUS", "outputColumn": "ZN", "useNext": 0, "column": [{"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}, {"Columndata": {"columnName": "INDUS", "type": "real", "min": "0.5", "max": "27.7", "mean": "11.1"}}, {"Columndata": {"columnName": "ZN", "type": "real", "min": "0.0", "max": "100.0", "mean": "11.4"}}]}]}], "fileheader": [{"field": "CRIM", "alias": "CRIM", "generated": 0, "position": 1, "type": "real"}, {"field": "ZN", "alias": "ZN", "generated": 0, "position": 2, "type": "real"}, {"field": "INDUS", "alias": "INDUS", "generated": 0, "position": 3, "type": "real"}, {"field": "CHAS", "alias": "CHAS", "generated": 0, "position": 4, "type": "real"}, {"field": "NOX", "alias": "NOX", "generated": 0, "position": 5, "type": "real"}, {"field": "RM", "alias": "RM", "generated": 0, "position": 6, "type": "real"}, {"field": "AGE", "alias": "AGE", "generated": 0, "position": 7, "type": "real"}, {"field": "DIS", "alias": "DIS", "generated": 0, "position": 8, "type": "real"}, {"field": "RAD", "alias": "RAD", "generated": 0, "position": 9, "type": "real"}, {"field": "TAX", "alias": "TAX", "generated": 0, "position": 10, "type": "real"}, {"field": "PTRATIO", "alias": "PTRATIO", "generated": 0, "position": 11, "type": "real"}, {"field": "B", "alias": "B", "generated": 0, "position": 12, "type": "real"}, {"field": "LSTAT", "alias": "LSTAT", "generated": 0, "position": 13, "type": "real"}, {"field": "MEDV", "alias": "MEDV", "generated": 0, "position": 14, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "CRIM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "2.33", "stddev": "5.05", "min": "0.01432", "max": "22.5971", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CRIM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ZN", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "17.85", "stddev": "20.34", "min": "2.46", "max": "101.32", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ZN"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INDUS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "9.55", "stddev": "5.79", "min": "1.32", "max": "19.58", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INDUS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CHAS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.04", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CHAS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NOX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "0.54", "stddev": "0.1", "min": "0.41100000000000003", "max": "0.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NOX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RM", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "6.48", "stddev": "0.77", "min": "5.0", "max": "8.725", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RM"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AGE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "72.41", "stddev": "24.8", "min": "17.2", "max": "98.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AGE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DIS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "3.76", "stddev": "2.09", "min": "1.5184", "max": "8.3248", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DIS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RAD", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "8.96", "stddev": "7.85", "min": "1.0", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RAD"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TAX", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "399.52", "stddev": "149.09", "min": "193.0", "max": "666.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TAX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PTRATIO", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "18.38", "stddev": "1.99", "min": "14.7", "max": "21.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PTRATIO"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "B", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "340.71", "stddev": "114.3", "min": "0.32", "max": "396.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "B"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LSTAT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "11.52", "stddev": "6.86", "min": "2.98", "max": "31.99", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "LSTAT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MEDV", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "25", "mean": "22.64", "stddev": "10.65", "min": "7.4", "max": "50.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MEDV"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run dataprepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "MEDV")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

